Python version: 3.9.2

In [1]:
from z3 import *
import numpy as np
import pandas as pd

In [2]:
# (row, column, value)

sample1 = [ # 4x4 board, 3 solutions
    (0,0,1), (0,1,2), 
    (1,1,4), (1,2,1), 
    (3,0,4), (3,2,2)
]

sample2 = sample1 + [(0,3,4)] # 4x4 board, 2 solutions

sample3 = [] # 4x4 board, 100+ solutions

sample4 = [ # 9x9 board, 1 solution
    (0,0,3), (0,2,5), (0,3,4), (0,5,2), (0,7,6), 
    (1,0,4), (1,1,9), (1,3,7), (1,4,6), (1,6,1), (1,8,8), 
    (2,0,6), (2,3,1), (2,5,3), (2,6,2), (2,7,4), (2,8,5), 
    (3,2,3), (3,3,9), (3,6,5), (3,7,8), 
    (4,0,9), (4,1,6), (4,4,5), (4,5,8), (4,6,7), (4,8,3), 
    (5,1,8), (5,2,1), (5,3,3), (5,5,4), (5,7,9), (5,8,2), 
    (6,1,5), (6,3,6), (6,5,1), (6,6,4), 
    (7,0,2), (7,3,5), (7,4,4), (7,5,9), (7,7,7), 
    (8,0,1), (8,1,4), (8,2,9), (8,5,7), (8,6,3), (8,8,6)
]

sample5 = sample4 + [(8,7,3)] # 3x3 unsolvable board

User inputs

In [3]:
N = 9
box = int(np.sqrt(N))

# True -> display every solution found
# False -> display only the last solution found
display_all_solutions = False

In [4]:
# (row, column, value)
constraints = sample4

Create and display the starting board

In [5]:
puzzle = pd.DataFrame(index=range(N),columns=range(N))
for i,j,k in constraints: 
    puzzle.iloc[i,j] = k
puzzle = puzzle.fillna(0)
print("Starting board: (zeros indicate unfilled cells)")
display(puzzle)
puzzle -= 1 # zero-index

Starting board: (zeros indicate unfilled cells)


,0,1,2,3,4,5,6,7,8
0,3,0,5,4,0,2,0,6,0
1,4,9,0,7,6,0,1,0,8
2,6,0,0,1,0,3,2,4,5
3,0,0,3,9,0,0,5,8,0
4,9,6,0,0,5,8,7,0,3
5,0,8,1,3,0,4,0,9,2
6,0,5,0,6,0,1,4,0,0
7,2,0,0,5,4,9,0,7,0
8,1,4,9,0,0,7,3,0,6


Create Boolean operators and starting puzzle constraints. `C_ijk = True` iff entry at row `i`, col `j` has value `k+1`

In [6]:
s = Solver()

In [7]:
bools = pd.DataFrame(index=range(N),columns=range(N))
for i in range(N): 
    for j in range(N):
        bools.iloc[i,j] = BoolVector('C_' + str(i) + str(j), N)
        if puzzle.iloc[i,j] >= 0: 
            s.add(bools.iloc[i,j][puzzle.iloc[i,j]])

Add generic sudoku constraints

In [8]:
for i in range(N):
    for j in range(N):
        # Each cell must have exactly 1 value
        s.add(Or(bools.iloc[i,j])) 
        # each row has at least one 1, one 2, etc
        s.add(Or([bools.iloc[i,k][j] for k in range(N)])) 
        # each column has at least one 1, one 2, etc
        s.add(Or([bools.iloc[k,i][j] for k in range(N)]))

In [9]:
# each box has at least one 1, one 2, etc 
for r in range(box):
    for c in range(box):
        for k in range(N):
            s.add(Or([bools.iloc[box*r+i, box*c+j][k] for i in range(box) for j in range(box)]))

In [10]:
# no cell has multiple values
for i in range(N):
    for j in range(N):
        s.add(And([Not(And(bools.iloc[i,j][k],bools.iloc[i,j][l])) for k in range(N) for l in range(k)]))

Solve the problem, count unique solutions

In [11]:
def solution_board(model, bools):
    '''
    return the solution board (numbers on board are 1-indexed)
    '''
    solution = pd.DataFrame(index=range(N),columns=range(N))
    for i in range(N): 
        for j in range(N): 
            for k in range(N):
                if model.eval(bools.iloc[i,j][k]): 
                    solution.iloc[i,j] = k + 1
                    break
    return solution

In [12]:
def invalidate(solution, bools): 
    '''
    return a clause to exclude solution
    '''
    exclude = []
    for i in range(N): 
        for j in range(N): 
            exclude.append(Not(bools.iloc[i,j][solution.iloc[i,j]-1]))
    return Or(exclude)

In [13]:
# this cell only works the first time you run it.
num_solutions = 0
while (num_solutions < 100) and (s.check() == sat): 
    num_solutions += 1
    model = s.model()
    board = solution_board(model, bools)
    if display_all_solutions: 
        display(board)
    s.add(invalidate(board, bools)) # exclude this solution
if not num_solutions: 
    print("The board is not solvable.")
elif num_solutions == 1: 
    print("Unique solution found!")
elif num_solutions >= 100: 
    print('More than 100 (nonunique) solutions found!\n')
else: 
    print(num_solutions, '(nonunique) solutions found!\n')
    
if num_solutions: 
    print('Here is a solution:')
    display(board)

Unique solution found!
Here is a solution:


,0,1,2,3,4,5,6,7,8
0,3,1,5,4,8,2,9,6,7
1,4,9,2,7,6,5,1,3,8
2,6,7,8,1,9,3,2,4,5
3,7,2,3,9,1,6,5,8,4
4,9,6,4,2,5,8,7,1,3
5,5,8,1,3,7,4,6,9,2
6,8,5,7,6,3,1,4,2,9
7,2,3,6,5,4,9,8,7,1
8,1,4,9,8,2,7,3,5,6


In [14]:
print('Original board for reference:')
display((puzzle + 1).replace(0,''))

Original board for reference:


,0,1,2,3,4,5,6,7,8
0,3,,5,4,,2,,6,
1,4,9,,7,6,,1,,8
2,6,,,1,,3,2,4,5
3,,,3,9,,,5,8,
4,9,6,,,5,8,7,,3
5,,8,1,3,,4,,9,2
6,,5,,6,,1,4,,
7,2,,,5,4,9,,7,
8,1,4,9,,,7,3,,6
